### Import libraries

In [21]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [22]:
train = pd.read_excel('Data_Train.xlsx')
test = pd.read_excel('Test_set.xlsx')

In [23]:
train.shape, test.shape

((10683, 11), (2671, 10))

In [24]:
train.info()
#test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10683 entries, 0 to 10682
Data columns (total 11 columns):
Airline            10683 non-null object
Date_of_Journey    10683 non-null object
Source             10683 non-null object
Destination        10683 non-null object
Route              10682 non-null object
Dep_Time           10683 non-null object
Arrival_Time       10683 non-null object
Duration           10683 non-null object
Total_Stops        10682 non-null object
Additional_Info    10683 non-null object
Price              10683 non-null int64
dtypes: int64(1), object(10)
memory usage: 918.2+ KB


In [25]:
train.head(10)

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302
5,SpiceJet,24/06/2019,Kolkata,Banglore,CCU → BLR,09:00,11:25,2h 25m,non-stop,No info,3873
6,Jet Airways,12/03/2019,Banglore,New Delhi,BLR → BOM → DEL,18:55,10:25 13 Mar,15h 30m,1 stop,In-flight meal not included,11087
7,Jet Airways,01/03/2019,Banglore,New Delhi,BLR → BOM → DEL,08:00,05:05 02 Mar,21h 5m,1 stop,No info,22270
8,Jet Airways,12/03/2019,Banglore,New Delhi,BLR → BOM → DEL,08:55,10:25 13 Mar,25h 30m,1 stop,In-flight meal not included,11087
9,Multiple carriers,27/05/2019,Delhi,Cochin,DEL → BOM → COK,11:25,19:15,7h 50m,1 stop,No info,8625


In [26]:
for i in train.columns:
    print("Unique Values:", i, train[i].nunique())

Unique Values: Airline 12
Unique Values: Date_of_Journey 44
Unique Values: Source 5
Unique Values: Destination 6
Unique Values: Route 128
Unique Values: Dep_Time 222
Unique Values: Arrival_Time 1343
Unique Values: Duration 368
Unique Values: Total_Stops 5
Unique Values: Additional_Info 10
Unique Values: Price 1870


In [27]:
for i in test.columns:
    print("Unique Values:", i, test[i].nunique())

Unique Values: Airline 11
Unique Values: Date_of_Journey 44
Unique Values: Source 5
Unique Values: Destination 6
Unique Values: Route 100
Unique Values: Dep_Time 199
Unique Values: Arrival_Time 704
Unique Values: Duration 320
Unique Values: Total_Stops 5
Unique Values: Additional_Info 6


### Set up dataframe

In [28]:
train_df = train[['Airline', 'Source', 'Destination', 'Total_Stops', 'Additional_Info', 'Date_of_Journey', 'Dep_Time', 'Duration',
                  'Route', 'Arrival_Time', 'Price']]
test_df = test[['Airline', 'Source', 'Destination', 'Total_Stops', 'Additional_Info', 'Date_of_Journey', 'Dep_Time', 
                'Duration',
                'Route', 'Arrival_Time']]

### Adding Features
Different classes have different price. Since information is not given specifically and from inspection we can assume majority of airlines are economy.All are economy except for a few airlines

In [29]:
Class = {'IndiGo': 'Economy',
         'GoAir': 'Economy',
         'Vistara': 'Economy',
         'Vistara Premium economy': 'Premium Economy',
         'Air Asia': 'Economy',
         'Trujet': 'Economy',
         'Jet Airways': 'Economy',
         'SpiceJet': 'Economy',
         'Jet Airways Business': 'Business',
         'Air India': 'Economy',
         'Multiple carriers': 'Economy',
         'Multiple carriers Premium economy': 'Premium Economy'}
train_df['Booking_Class'] = train_df['Airline'].map(Class)
test_df['Booking_Class'] = test_df['Airline'].map(Class)

MarketShare of each airline is important since different shares have difference of power in market. Information is taken mostly from Wikipedia. For Multiple carriers & Multiple carriers Premium Economy I have assumed 1% & for the Trujet which is a new airline business I have assumed 0.1%.

In [30]:
market = {'IndiGo': 41.3,
         'GoAir': 8.4,
         'Vistara': 3.3,
         'Vistara Premium economy': 3.3,
         'Air Asia': 3.3,
         'Trujet': 0.1,
         'Jet Airways': 17.8,
         'SpiceJet': 13.3,
         'Jet Airways Business': 17.8,
         'Air India': 13.5,
         'Multiple carriers': 1,
         'Multiple carriers Premium economy': 1}
train_df['Market_Share'] = train_df['Airline'].map(market)
test_df['Market_Share'] = test_df['Airline'].map(market)

We know how important it is to book in advance. Date of Booking is important. Since this information was not provided I have assumed 01-Mar-2019 as ticket booking date and created new feature Days_to_Departure

In [31]:

df1 = train_df.copy() 
df1['Day_of_Booking'] = '1/3/2019'
df1['Day_of_Booking'] = pd.to_datetime(df1['Day_of_Booking'],format='%d/%m/%Y')
df1['Date_of_Journey'] = pd.to_datetime(df1['Date_of_Journey'],format='%d/%m/%Y')
df1['Days_to_Departure'] = (df1['Date_of_Journey'] - df1['Day_of_Booking']).dt.days
train_df['Days_to_Departure'] = df1['Days_to_Departure']

df2 = test_df.copy() 
df2['Day_of_Booking'] = '1/3/2019'
df2['Day_of_Booking'] = pd.to_datetime(df2['Day_of_Booking'],format='%d/%m/%Y')
df2['Date_of_Journey'] = pd.to_datetime(df2['Date_of_Journey'],format='%d/%m/%Y')
df2['Days_to_Departure'] = (df2['Date_of_Journey'] - df2['Day_of_Booking']).dt.days
test_df['Days_to_Departure'] = df2['Days_to_Departure']

del df1, df2

Departure Time is very important. Price change throughout morning, noon, afternoon,evening,and night. Same should apply to Arrival Time. Creating new features: Depature Time of Day and Arrival Time of Day.

In [32]:
def get_departure(dep):
    dep = dep.split(':')
    dep = int(dep[0])
    if (dep >= 6 and dep < 12):
        return 'Morning'
    elif (dep >= 12 and dep < 17):
        return 'Noon'
    elif (dep >= 17 and dep < 20):
        return 'Evening'
    else:
        return 'Night'
    
train_df['Dep_timeofday'] = train['Dep_Time'].apply(get_departure)   
test_df['Dep_timeofday'] = test['Dep_Time'].apply(get_departure) 

train_df['Arr_timeofday'] = train['Arrival_Time'].apply(get_departure)   
test_df['Arr_timeofday'] = test['Arrival_Time'].apply(get_departure)

### Cleaning Numerical Features

Remove Date from Arrival time

In [33]:
train_df['Arrival_Time'] = train['Arrival_Time'].str.split(' ').str[0]
test_df['Arrival_Time'] = test['Arrival_Time'].str.split(' ').str[0]
train_df.head(2)

,Airline,Source,Destination,Total_Stops,Additional_Info,Date_of_Journey,Dep_Time,Duration,Route,Arrival_Time,Price,Booking_Class,Market_Share,Days_to_Departure,Dep_timeofday,Arr_timeofday
0,IndiGo,Banglore,New Delhi,non-stop,No info,24/03/2019,22:20,2h 50m,BLR → DEL,01:10,3897,Economy,41.3,23,Night,Night
1,Air India,Kolkata,Banglore,2 stops,No info,1/05/2019,05:50,7h 25m,CCU → IXR → BBI → BLR,13:15,7662,Economy,13.5,61,Night,Noon


Converted Total_Stops into float type

In [34]:
train_df['Total_Stops'] = train_df['Total_Stops'].astype(str)
train_df['Total_Stops'] = train_df['Total_Stops'].str.replace('non-stop','0')
train_df['Total_Stops'] = train_df['Total_Stops'].str.replace('stops','')
train_df['Total_Stops'] = train_df['Total_Stops'].str.replace('stop','')
train_df['Total_Stops'].fillna(0, inplace=True)   
train_df['Total_Stops'] = train_df['Total_Stops'].astype(float)

test_df['Total_Stops'] = test_df['Total_Stops'].astype(str)
test_df['Total_Stops'] = test_df['Total_Stops'].str.replace('non-stop','0')
test_df['Total_Stops'] = test_df['Total_Stops'].str.replace('stops','')
test_df['Total_Stops'] = test_df['Total_Stops'].str.replace('stop','')
test_df['Total_Stops'].fillna(0, inplace=True)
test_df['Total_Stops'] = test_df['Total_Stops'].astype(float)


Converted Duration into minutes only without hours units

In [35]:
train_df['Hours'] = train['Duration'].str.split(' ').str[0]
train_df['Hours'] = train_df['Hours'].str.replace('m','')
train_df['Hours'] = train_df['Hours'].str.replace('h','').astype(float)
train_df['Hours'].fillna(0, inplace=True) 

train_df['Minutes'] = train['Duration'].str.split(' ').str[1]
train_df['Minutes'] = train_df['Minutes'].str.replace('m','').astype(float)
train_df['Minutes'].fillna(0, inplace=True)

test_df['Hours'] = test['Duration'].str.split(' ').str[0]
test_df['Hours'] = test_df['Hours'].str.replace('m','')
test_df['Hours'] = test_df['Hours'].str.replace('h','').astype(float)
test_df['Hours'].fillna(0, inplace=True) 

test_df['Minutes'] = test['Duration'].str.split(' ').str[1]
test_df['Minutes'] = test_df['Minutes'].str.replace('m','').astype(float)
test_df['Minutes'].fillna(0, inplace=True)

train_df['Hours'] = train_df['Hours'] * 60
train_df['Duration'] = train_df['Hours'] + train_df['Minutes']

test_df['Hours'] = test_df['Hours'] * 60
test_df['Duration'] = test_df['Hours'] + test_df['Minutes']

train_df.drop(['Hours', 'Minutes'], axis=1, inplace=True)
test_df.drop(['Hours', 'Minutes'], axis=1, inplace=True)

Take logarithmic values of Price and Duration

In [36]:
train_df['Price'] = np.log1p(train_df['Price'])
train_df['Duration'] = np.log1p(train_df['Duration'])
test_df['Duration'] = np.log1p(test_df['Duration'])

Ugly Column with 10 unique, Drop it

In [37]:
train_df['Additional_Info'] = train_df.drop(columns='Additional_Info')
test_df['Additional_Info'] = test_df.drop(columns='Additional_Info')

### Cleaning Categorical Features for numerical models

For the Route column, I have applied TF-IDF text extraction to create one column for each value of location. There are 43 unique locations so 43 new features are created.

In [38]:
train_df = pd.get_dummies(train_df, columns=['Airline', 'Source', 'Destination', 'Additional_Info', 'Date_of_Journey',
                                             'Dep_Time', 'Arrival_Time', 'Dep_timeofday', 'Booking_Class', 'Arr_timeofday'],
                          drop_first=True)
test_df = pd.get_dummies(test_df, columns=['Airline', 'Source', 'Destination', 'Additional_Info', 'Date_of_Journey',
                                           'Dep_Time', 'Arrival_Time', 'Dep_timeofday', 'Booking_Class', 'Arr_timeofday'],
                         drop_first=True)
def clean_route(route):
    route = str(route)
    route = route.split(' → ')
    return ' '.join(route)

train_df['Route'] = train_df['Route'].apply(clean_route)
test_df['Route'] = test_df['Route'].apply(clean_route)

from sklearn.feature_extraction.text import TfidfVectorizer
tf = TfidfVectorizer(ngram_range=(1, 1), lowercase=False)
train_route = tf.fit_transform(train_df['Route'])
test_route = tf.transform(test_df['Route'])

train_route = pd.DataFrame(data=train_route.toarray(), columns=tf.get_feature_names())
test_route = pd.DataFrame(data=test_route.toarray(), columns=tf.get_feature_names())
train_route.head(5)

,AMD,ATQ,BBI,BDQ,BHO,BLR,BOM,CCU,COK,DED,...,PAT,PNQ,RPR,STV,TRV,UDR,VGA,VNS,VTZ,nan
0,0.0,0.0,0.000000,0.0,0.0,0.783012,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.619874,0.0,0.0,0.194833,0.000000,0.248235,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.000000,0.0,0.0,0.000000,0.241494,0.000000,0.271844,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.000000,0.0,0.0,0.274089,0.000000,0.349214,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.000000,0.0,0.0,0.284914,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Add these dataframes into train_df and test_df

In [39]:
train_df = pd.concat([train_df, train_route], axis=1) 
train_df.drop('Route', axis=1, inplace=True)

test_df = pd.concat([test_df, test_route], axis=1) 
test_df.drop('Route', axis=1, inplace=True)

In [40]:
train_df.head()

,Total_Stops,Duration,Price,Market_Share,Days_to_Departure,Airline_Air India,Airline_GoAir,Airline_IndiGo,Airline_Jet Airways,Airline_Jet Airways Business,...,PAT,PNQ,RPR,STV,TRV,UDR,VGA,VNS,VTZ,nan
0,0.0,5.141664,8.268219,41.3,23,0,0,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2.0,6.100319,8.944159,13.5,61,1,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2.0,7.039660,9.538420,17.8,100,0,0,0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,5.786897,8.735364,41.3,72,0,0,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,5.655992,9.495745,41.3,0,0,0,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Train Split

In [41]:
X = train_df.drop(labels=['Price'], axis=1)
y = train_df['Price'].values

from sklearn.model_selection import train_test_split
X_train, X_cv, y_train, y_cv = train_test_split(X, y, test_size=0.25, random_state=1)

X_train.shape, y_train.shape, X_cv.shape, y_cv.shape

((8012, 572), (8012,), (2671, 572), (2671,))

In [52]:
train_df.columns = ["".join (c if c.isalnum() else "_" for c in str(x)) for x in train_df.columns]
test_df.columns = ["".join (c if c.isalnum() else "_" for c in str(x)) for x in test_df.columns]

### Model Building

In [53]:
# Import libraries
from math import sqrt 
from sklearn.metrics import mean_squared_log_error

In [ ]:
train_df.columns = ["".join (c if c.isalnum() else "_" for c in str(x)) for x in train_df.columns]
import lightgbm as lgb
train_data = lgb.Dataset(X_train, label=y_train)
test_data = lgb.Dataset(X_cv, label=y_cv)

param = {'objective': 'regression',
         'boosting': 'gbdt',
         'num_iterations': 3000,   
         'learning_rate': 0.06,  
         'num_leaves': 40,  
         'max_depth': 24,   
         'min_data_in_leaf':11,  
         'max_bin': 4, 
         'metric': 'l2_root'
         }

lgbm = lgb.train(params=param,
                 verbose_eval=1000,
                 train_set=train_data,
                 valid_sets=[test_data])

y_pred2 = lgbm.predict(X_cv)
print('RMSLE:', sqrt(mean_squared_log_error(np.exp(y_cv), np.exp(y_pred2))))

In [ ]:
from xgboost import XGBRegressor
xgb = XGBRegressor(max_depth=9, 
                   learning_rate=0.5, 
                   n_estimators=112, 
                   silent=False, 
                   objective='reg:linear', 
                   booster='gbtree', 
                   n_jobs=1, 
                   nthread=None, 
                   gamma=0, 
                   min_child_weight=1, 
                   max_delta_step=0, 
                   subsample=1, 
                   colsample_bytree=1, 
                   colsample_bylevel=1, 
                   reg_alpha=1, 
                   reg_lambda=1, 
                   scale_pos_weight=1, 
                   base_score=0.5, 
                   random_state=0, 
                   seed=None)
xgb.fit(X_train, y_train)
y_pred1 = xgb.predict(X_cv)
print('RMSLE:', sqrt(mean_squared_log_error(np.exp(y_cv), np.exp(y_pred1))))
#RMSLE:0.11124007381703037

In [ ]:
from sklearn.ensemble import BaggingRegressor
br = BaggingRegressor(base_estimator=None, 
                      n_estimators=50, 
                      max_samples=1.0, 
                      max_features=1.0, 
                      bootstrap=True, 
                      bootstrap_features=False, 
                      oob_score=False, 
                      warm_start=False, 
                      n_jobs=1, 
                      random_state=1, 
                      verbose=0)
br.fit(X_train, y_train)
y_pred3 = br.predict(X_cv)
print('RMSLE:', sqrt(mean_squared_log_error(np.exp(y_cv), np.exp(y_pred3))))
#RMSLE:0.11265336177662688

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
gb = GradientBoostingRegressor(loss='ls', 
                               learning_rate=0.3, 
                               n_estimators=380, 
                               subsample=1.0, 
                               criterion='friedman_mse', 
                               min_samples_split=30, 
                               min_samples_leaf=1, 
                               min_weight_fraction_leaf=0.0, 
                               max_depth=7, 
                               min_impurity_decrease=0.0, 
                               min_impurity_split=None, 
                               init=None, 
                               random_state=0, 
                               max_features=None, 
                               alpha=0.9, 
                               verbose=100, 
                               max_leaf_nodes=None, 
                               warm_start=False, 
                               presort='auto')
gb.fit(X_train, y_train)
y_pred4 = gb.predict(X_cv)
print('RMSLE:', sqrt(mean_squared_log_error(np.exp(y_cv), np.exp(y_pred4))))
#RMSLE:0.11014737617371691

In [ ]:
y_pred = y_pred1*0.10 + y_pred2*0.50 + y_pred3*0.20 + y_pred4*0.20
print('RMSLE:', sqrt(mean_squared_log_error(np.exp(y_cv), np.exp(y_pred))))

### Test Set Prediction

In [177]:
train_df['Dep_Time_22:30'] = 0

In [178]:
missing_cols_test = []
for col in train_df.columns:
    if col not in test_df.columns:
        missing_cols_test.append(col)
        
for i in missing_cols_test:
    test_df[i] = 0

test_df.drop('Price', axis=1, inplace=True)


In [179]:
train_df = train_df.reindex(sorted(train_df.columns), axis=1)
test_df = test_df.reindex(sorted(test_df.columns), axis=1)

In [180]:

X_train = train_df.drop(labels='Price', axis=1)
y_train = train_df['Price'].values

X_test = test_df

In [181]:
X_train.shape, X_test.shape

((10683, 573), (2671, 573))

In [ ]:
from xgboost import XGBRegressor
xgb = XGBRegressor(max_depth=9, 
                   learning_rate=0.5, 
                   n_estimators=112, 
                   silent=False, 
                   objective='reg:linear', 
                   booster='gbtree', 
                   n_jobs=1, 
                   nthread=None, 
                   gamma=0, 
                   min_child_weight=1, 
                   max_delta_step=0, 
                   subsample=1, 
                   colsample_bytree=1, 
                   colsample_bylevel=1, 
                   reg_alpha=0.89, 
                   reg_lambda=1, 
                   scale_pos_weight=1, 
                   base_score=0.5, 
                   random_state=0, 
                   seed=None)
xgb.fit(X_train, y_train)
y_pred1 = xgb.predict(X_test)

In [ ]:
import lightgbm as lgb
train_data = lgb.Dataset(X_train, label=y_train)

param = {'objective': 'regression',
         'boosting': 'gbdt',
         'num_iterations': 3000,   
         'learning_rate': 0.06,  
         'num_leaves': 40,  
         'max_depth': 24,   
         'min_data_in_leaf':11,  
         'max_bin': 4, 
         'metric': 'l2_root'
         }

lgbm = lgb.train(params=param,
                 train_set=train_data)

y_pred2 = lgbm.predict(X_test)

In [ ]:
from sklearn.ensemble import BaggingRegressor
br = BaggingRegressor(base_estimator=None, 
                      n_estimators=50, 
                      max_samples=1.0, 
                      max_features=1.0, 
                      bootstrap=True, 
                      bootstrap_features=False, 
                      oob_score=False, 
                      warm_start=False, 
                      n_jobs=1, 
                      random_state=1, 
                      verbose=0)
br.fit(X_train, y_train)
y_pred3 = br.predict(X_test)

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
gb = GradientBoostingRegressor(loss='ls', 
                               learning_rate=0.3, 
                               n_estimators=380, 
                               subsample=1.0, 
                               criterion='friedman_mse', 
                               min_samples_split=30, 
                               min_samples_leaf=1, 
                               min_weight_fraction_leaf=0.0, 
                               max_depth=7, 
                               min_impurity_decrease=0.0, 
                               min_impurity_split=None, 
                               init=None, 
                               random_state=0, 
                               max_features=None, 
                               alpha=0.9, 
                               verbose=100, 
                               max_leaf_nodes=None, 
                               warm_start=False, 
                               presort='auto')
gb.fit(X_train, y_train)
y_pred4 = gb.predict(X_test)

In [ ]:
y_pred = y_pred1*0.15 + y_pred2*0.50 + y_pred3*0.15 + y_pred4*0.20
y_pred = np.exp(y_pred)